# Orange Juice Toy Problem C

This toy problem illustrates how we can use the abstract model built into the `mola` package in order to solve an optimisation problem based on openLCA data.

## Problem Statement

A citrus fruit juice producer in wishes to produce 100kg of fresh orange juice in either the US, South Africa or RoW. In order to produce the juice, they need to source local oranges and transport them to their local processing plant. It takes 2 kg of oranges produce 1 kg of orange juice. The distance between fruit production and processing plant in each country is shown in the table below. The options for freight transport in each country are diesel train, electric train, 16-32 ton lorry EURO1 and 16-32 ton lorry EURO2. 

They would like to choose where to produce orange juice and the means of transport of oranges in order to minimise the environmental impact of green house gases.

Country | Distance / km
-- | --
US | 200
South Africa | 600
RoW | 400

Assumptions
* The distance travelled is independent of the mode of transport.
* The environmental impact of converting oranges to juice is the same in each country.
* If relevant openLCA data does not exist then we neither use a proxy nor create a new system process.

## Specification

In this notebook we use the full model to solve the orange juice problem in Problem B using the abstract specification v5.



In [1]:
import mola.specification5 as ms
from importlib import reload
spec = ms.ScheduleSpecification()
spec.abstract_model.AP.pprint()

AP : All processes from in OpenLCA database
    Size=0, Index=None, Ordered=Insertion
    Not constructed


# User Configuration

If there is no existing user set configuration then create a new empty model set configuration with the abstract model specification.

In [2]:
import mola.pyomoio as pyoio
model_set_file_name = 'model_set_data.json'
user_sets = pyoio.get_model_user_sets(spec, model_set_file_name)
user_sets

Model sets saved


{'F_m': ['1f7bbd3e-fcd1-412d-8608-035b855ea735'],
 'F_s': [],
 'F_t': ['0ace02fa-eca5-482d-a829-c18e46a52db4',
  '4ecf1190-7028-4038-89b0-79adeb3e98cc',
  '559d5695-102a-4beb-9f7f-3db334c9b51d'],
 'D': ['d1'],
 'T': ['t1'],
 'K': ['k1'],
 'P_m': ['f22f5f6e-1bdc-3cb5-8f48-8a04d8f9b768',
  'cfc3f58f-7bdf-3019-888b-c4f2dbc332e8',
  '760f1858-b0ef-3280-930e-88ebe4645061'],
 'P_t': ['44ad59ca-4fe0-394c-a6d9-5dea68783c23',
  'f615ae37-fa6d-4a47-958c-23234c986400',
  '9106bf8d-869d-425f-b8ee-0bf9a78bcf65',
  '97aa2533-ae98-3f8f-b3e2-3acb9d66a010',
  'b37252a8-fbb5-47cb-a3aa-1485d090bdfd',
  '660ba3fe-d7a5-4c43-8777-27d331934edf',
  'fe7d1840-4341-40fd-ba6d-fbba126ca7d3'],
 'P_s': [],
 'KPI': ['061b7db5-4f56-3368-bf50-9ff0fcc8dd1f'],
 'OBJ': ['environment', 'cost'],
 'F': [],
 'P': []}

## Sets

To configure the model we need lookup tables to select sets. These come from a database so part of the new model configuration is a link to a database.

In [3]:
import mola.dataview as dv
import mola.dataimport as di
import pandas as pd
from pyomo.environ import *
conn = di.get_sqlite_connection()
lookup = dv.get_lookup_tables(conn)

SELECT "REF_ID","NAME" FROM "TBL_CATEGORIES"
SELECT "REF_ID" "FLOW_REF_ID","NAME" FROM "TBL_FLOWS"
SELECT "TBL_PROCESSES"."REF_ID" "PROCESS_REF_ID","TBL_PROCESSES"."NAME" "PROCESS_NAME","TBL_LOCATIONS"."NAME" "LOCATION_NAME" FROM "TBL_PROCESSES" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID"
SELECT "REF_ID","NAME" FROM "TBL_FLOWS" WHERE "FLOW_TYPE"='PRODUCT_FLOW'
SELECT "TBL_IMPACT_METHODS"."NAME" "method_NAME","TBL_IMPACT_CATEGORIES"."REF_ID" "REF_ID","TBL_IMPACT_CATEGORIES"."NAME" "category_NAME" FROM "TBL_IMPACT_CATEGORIES" LEFT JOIN "TBL_IMPACT_METHODS" ON "TBL_IMPACT_CATEGORIES"."F_IMPACT_METHOD"="TBL_IMPACT_METHODS"."ID"


We show a widget that loads and saves the user configuration of sets in a tabbed interface. Existing user configuration data is loaded if there is a saved model data file.

In [4]:
# any way to distinquish material flows from transport flows in openLCA? fix a group of categories in lookup?
import mola.widgets as mw
reload(mw)
lookups = {'F_m': lookup['flows'], 'P_m': lookup['processes'], 'F_t': lookup['flows'], 
           'P_t': lookup['processes'], 'F_s': lookup['flows'], 
           'P_s': lookup['processes'], 'KPI': lookup['KPI']}
vbox, tab = mw.get_sets(spec, lookups, model_set_file_name)
vbox

The content of the json set file is shown below.

In [5]:
import json
with open(model_set_file_name) as s:
    set_data = json.load(s)
set_data

{'F_m': ['1f7bbd3e-fcd1-412d-8608-035b855ea735'],
 'F_s': [],
 'F_t': ['0ace02fa-eca5-482d-a829-c18e46a52db4',
  '4ecf1190-7028-4038-89b0-79adeb3e98cc',
  '559d5695-102a-4beb-9f7f-3db334c9b51d'],
 'D': ['d1'],
 'T': ['t1'],
 'K': ['k1'],
 'P_m': ['f22f5f6e-1bdc-3cb5-8f48-8a04d8f9b768',
  'cfc3f58f-7bdf-3019-888b-c4f2dbc332e8',
  '760f1858-b0ef-3280-930e-88ebe4645061'],
 'P_t': ['44ad59ca-4fe0-394c-a6d9-5dea68783c23',
  'f615ae37-fa6d-4a47-958c-23234c986400',
  '9106bf8d-869d-425f-b8ee-0bf9a78bcf65',
  '97aa2533-ae98-3f8f-b3e2-3acb9d66a010',
  'b37252a8-fbb5-47cb-a3aa-1485d090bdfd',
  '660ba3fe-d7a5-4c43-8777-27d331934edf',
  'fe7d1840-4341-40fd-ba6d-fbba126ca7d3'],
 'P_s': [],
 'KPI': ['061b7db5-4f56-3368-bf50-9ff0fcc8dd1f'],
 'OBJ': ['environment', 'cost'],
 'F': [],
 'P': []}

## Parameters

The number of model parameters depends on the elements in the sets. At the moment, there is a DataFrame interface for setting these parameters. The current parameter set held in a file is restored if set combinations in its definition are still applicable.

In [6]:
import qgrid
reload(pyoio)
reload(mw)
model_parameters_file_name = 'model_parameters_data.json'
param_dfr, param_dict = pyoio.get_model_user_parameters(spec, set_data, model_parameters_file_name)
mw.get_parameters(param_dfr, model_parameters_file_name)

Save the parameters above to the file system to persist any changes before proceeding. For reference here is the contents of the model parameter file.

In [7]:
with open(model_parameters_file_name) as fp:
    params_json = json.load(fp)
params_json

{'C': [{'index': ['1f7bbd3e-fcd1-412d-8608-035b855ea735', 'k1', 'd1', 't1'],
   'value': 0.5}],
 'Demand': [{'index': ['d1', 'k1', 't1'], 'value': 0.0}],
 'J': [{'index': ['1f7bbd3e-fcd1-412d-8608-035b855ea735',
    'f22f5f6e-1bdc-3cb5-8f48-8a04d8f9b768',
    '0ace02fa-eca5-482d-a829-c18e46a52db4',
    '44ad59ca-4fe0-394c-a6d9-5dea68783c23'],
   'value': 0.0},
  {'index': ['1f7bbd3e-fcd1-412d-8608-035b855ea735',
    'f22f5f6e-1bdc-3cb5-8f48-8a04d8f9b768',
    '0ace02fa-eca5-482d-a829-c18e46a52db4',
    'f615ae37-fa6d-4a47-958c-23234c986400'],
   'value': 0.0},
  {'index': ['1f7bbd3e-fcd1-412d-8608-035b855ea735',
    'f22f5f6e-1bdc-3cb5-8f48-8a04d8f9b768',
    '0ace02fa-eca5-482d-a829-c18e46a52db4',
    '9106bf8d-869d-425f-b8ee-0bf9a78bcf65'],
   'value': 1.0},
  {'index': ['1f7bbd3e-fcd1-412d-8608-035b855ea735',
    'f22f5f6e-1bdc-3cb5-8f48-8a04d8f9b768',
    '0ace02fa-eca5-482d-a829-c18e46a52db4',
    '97aa2533-ae98-3f8f-b3e2-3acb9d66a010'],
   'value': 0.0},
  {'index': ['1f7bbd3e-fc

# Model Build

To build the model we need to calculate the environmental impacts for each flow, process and KPI from the database. This is done as part in the `populate` method.

In [8]:
json_files = ['model_set_data.json', model_parameters_file_name]
#model_instance = spec.populate(db_file, json_files, elementary_flow_ref_ids=['e1', 'e2'])
model_instance = spec.populate(json_files)
model_instance.F_m.pprint()

F_m : Material flows to optimise
    Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    1 : {'1f7bbd3e-fcd1-412d-8608-035b855ea735',}


In the following sections we illustrate how the problem data has populated the abstract model to generate a concrete model instance.

## Sets

In [9]:
sets_dfr = pd.DataFrame(
    ([v.name, v.doc, len(v)] for v in model_instance.component_objects(Set, active=True)),
    columns=['Set', 'Description', 'Number of elements']
)
sets_dfr

,Set,Description,Number of elements
0,P_m,Processes producing material flows in the opti...,3
1,P_t,Processes producing transport flows in the opt...,7
2,P_s,Processes producing service flows in the optim...,0
3,F_m,Material flows to optimise,1
4,F_t,Transport flows to optimise,3
5,F_s,Service flows to optimise,0
6,T,Time intervals,1
7,K,Tasks,1
8,D,Demands,1
9,KPI,Performance indicators for optimisation problem,1


## Parameters

In [10]:
param_dfr = pd.DataFrame(
    ([o.name, o.doc, len(o), [index for index in o], [value(o[index]) for index in o]] for o in model_instance.component_objects(Param, active=True)),
    columns=['Param', 'Description', 'Number of elements', 'Dimension', 'Value']
)
qgrid.show_grid(param_dfr)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## Constraints

In [11]:
import qgrid
dfr = pd.DataFrame(
    ([v.name, v.expr] for v in model_instance.component_data_objects(Constraint, active=True)),
    columns=['Constraint', 'Expression']
)
qgrid.show_grid(dfr)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [12]:
print(dfr.iloc[0,0])
print(dfr.iloc[0,1])

total_demand_constraint[d1,k1]
Total_Demand[d1,k1]  <=  C[1f7bbd3e-fcd1-412d-8608-035b855ea735,k1,d1,t1]*Flow[1f7bbd3e-fcd1-412d-8608-035b855ea735,f22f5f6e-1bdc-3cb5-8f48-8a04d8f9b768,k1,t1] + C[1f7bbd3e-fcd1-412d-8608-035b855ea735,k1,d1,t1]*Flow[1f7bbd3e-fcd1-412d-8608-035b855ea735,cfc3f58f-7bdf-3019-888b-c4f2dbc332e8,k1,t1] + C[1f7bbd3e-fcd1-412d-8608-035b855ea735,k1,d1,t1]*Flow[1f7bbd3e-fcd1-412d-8608-035b855ea735,760f1858-b0ef-3280-930e-88ebe4645061,k1,t1]


## Objectives

In [13]:
dfr = pd.DataFrame(
    ([v.name, v.expr] for v in model_instance.component_data_objects(Objective, active=True)),
    columns=['Objective', 'Expression']
)

qgrid.show_grid(dfr)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

# Solver

We need to activate an objective or form a weighted sum of objectives. For the orange problem we just need the first objective of minimising the environmental impact.

In [14]:
model_instance.obj1.activate()
model_instance.obj2.deactivate()
model_instance.obj.deactivate()

In [15]:
opt = SolverFactory("glpk")
results = opt.solve(model_instance)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2235.00606936757
  Upper bound: 2235.00606936757
  Number of objectives: 1
  Number of constraints: 28
  Number of variables: 33
  Number of nonzeros: 46
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.020490169525146484
# ----------------------------------------------------------
#   Solution Information
# -----------

The variable output is shown below

In [16]:
model_instance.Flow.pprint()

Flow : Material flow
    Size=3, Index=Flow_index
    Key                                                                                          : Lower : Value : Upper : Fixed : Stale : Domain
    ('1f7bbd3e-fcd1-412d-8608-035b855ea735', '760f1858-b0ef-3280-930e-88ebe4645061', 'k1', 't1') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('1f7bbd3e-fcd1-412d-8608-035b855ea735', 'cfc3f58f-7bdf-3019-888b-c4f2dbc332e8', 'k1', 't1') :     0 : 200.0 :  None : False : False : NonNegativeReals
    ('1f7bbd3e-fcd1-412d-8608-035b855ea735', 'f22f5f6e-1bdc-3cb5-8f48-8a04d8f9b768', 'k1', 't1') :     0 :   0.0 :  None : False : False : NonNegativeReals


In [17]:
model_instance.Specific_Material_Transport_Flow.pprint()

Specific_Material_Transport_Flow : Specific Material Transport Flow
    Size=63, Index=Specific_Material_Transport_Flow_index
    Key                                                                                                                                                                          : Lower : Value : Upper : Fixed : Stale : Domain
    ('1f7bbd3e-fcd1-412d-8608-035b855ea735', '760f1858-b0ef-3280-930e-88ebe4645061', '0ace02fa-eca5-482d-a829-c18e46a52db4', '44ad59ca-4fe0-394c-a6d9-5dea68783c23', 'k1', 't1') :     0 :  None :  None : False :  True : NonNegativeReals
    ('1f7bbd3e-fcd1-412d-8608-035b855ea735', '760f1858-b0ef-3280-930e-88ebe4645061', '0ace02fa-eca5-482d-a829-c18e46a52db4', '660ba3fe-d7a5-4c43-8777-27d331934edf', 'k1', 't1') :     0 :  None :  None : False :  True : NonNegativeReals
    ('1f7bbd3e-fcd1-412d-8608-035b855ea735', '760f1858-b0ef-3280-930e-88ebe4645061', '0ace02fa-eca5-482d-a829-c18e46a52db4', '9106bf8d-869d-425f-b8ee-0bf9a78bcf65', 'k1', 't1')

In [18]:
model_instance.Specific_Transport_Flow.pprint()

Specific_Transport_Flow : Specific Transport Flow
    Size=21, Index=Specific_Transport_Flow_index
    Key                                                                                          : Lower : Value   : Upper : Fixed : Stale : Domain
    ('0ace02fa-eca5-482d-a829-c18e46a52db4', '44ad59ca-4fe0-394c-a6d9-5dea68783c23', 'k1', 't1') :     0 : 40000.0 :  None : False : False : NonNegativeReals
    ('0ace02fa-eca5-482d-a829-c18e46a52db4', '660ba3fe-d7a5-4c43-8777-27d331934edf', 'k1', 't1') :     0 :     0.0 :  None : False : False : NonNegativeReals
    ('0ace02fa-eca5-482d-a829-c18e46a52db4', '9106bf8d-869d-425f-b8ee-0bf9a78bcf65', 'k1', 't1') :     0 :     0.0 :  None : False : False : NonNegativeReals
    ('0ace02fa-eca5-482d-a829-c18e46a52db4', '97aa2533-ae98-3f8f-b3e2-3acb9d66a010', 'k1', 't1') :     0 :     0.0 :  None : False : False : NonNegativeReals
    ('0ace02fa-eca5-482d-a829-c18e46a52db4', 'b37252a8-fbb5-47cb-a3aa-1485d090bdfd', 'k1', 't1') :     0 :     0.0 :  Non

In [19]:
model_instance.transport_constraint.pprint()

transport_constraint : Size=21, Index=transport_constraint_index, Active=True
    Key                                                                                          : Lower : Body                                                                                                                                                                                                                                                                                                                                                                                       : Upper : Active
    ('0ace02fa-eca5-482d-a829-c18e46a52db4', '44ad59ca-4fe0-394c-a6d9-5dea68783c23', 'k1', 't1') :   0.0 : Specific_Transport_Flow[0ace02fa-eca5-482d-a829-c18e46a52db4,44ad59ca-4fe0-394c-a6d9-5dea68783c23,k1,t1] - dd[cfc3f58f-7bdf-3019-888b-c4f2dbc332e8,1f7bbd3e-fcd1-412d-8608-035b855ea735,k1,t1]*Specific_Material_Transport_Flow[1f7bbd3e-fcd1-412d-8608-035b855ea735,cfc3f58f-7bdf-3019-888b-c4f2dbc332e8,0ace02fa-eca5-

# Prettified output

We use the `mola.output` module to prettify the output.

In [20]:
import mola.output as mo
lookup = dv.LookupTables(conn)

SELECT "REF_ID","NAME" FROM "TBL_CATEGORIES"
SELECT "REF_ID" "FLOW_REF_ID","NAME" FROM "TBL_FLOWS"
SELECT "TBL_PROCESSES"."REF_ID" "PROCESS_REF_ID","TBL_PROCESSES"."NAME" "PROCESS_NAME","TBL_LOCATIONS"."NAME" "LOCATION_NAME" FROM "TBL_PROCESSES" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID"
SELECT "REF_ID","NAME" FROM "TBL_FLOWS" WHERE "FLOW_TYPE"='PRODUCT_FLOW'
SELECT "TBL_IMPACT_METHODS"."NAME" "method_NAME","TBL_IMPACT_CATEGORIES"."REF_ID" "REF_ID","TBL_IMPACT_CATEGORIES"."NAME" "category_NAME" FROM "TBL_IMPACT_CATEGORIES" LEFT JOIN "TBL_IMPACT_METHODS" ON "TBL_IMPACT_CATEGORIES"."F_IMPACT_METHOD"="TBL_IMPACT_METHODS"."ID"


In [21]:
flows_dfr = mo.get_entity(model_instance.component('Flow'), lookup).reset_index(drop=True)
qgrid.show_grid(flows_dfr)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [22]:
f_dfr = mo.get_entity(model_instance.component('Specific_Material_Transport_Flow'), lookup).reset_index(drop=True).dropna()
qgrid.show_grid(f_dfr[['Specific_Material_Transport_Flow', 'P_t']])

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [23]:
t_dfr = mo.get_entity(model_instance.component('Specific_Transport_Flow'), lookup).reset_index(drop=True).dropna()
qgrid.show_grid(t_dfr[['Specific_Transport_Flow', 'P_t']])

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…